# Exam
1. Load the dataset 
https://huggingface.co/datasets/poem_sentiment
2. Set 'bert-base-uncased' tokenizer and model for your task.
3. Tokenize data and set train/test/validation data
4. Create Trainer with the correct configuration
5. Use Trainer and fine-tune your model for 5 epochs.
6. Predict on test data and report accuracy.


In [20]:
# IMPORTANT
!pip install -q transformers==4.28.0
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 62.9 MB/s eta 0:00:00


In [21]:
from datasets import load_dataset, DatasetDict

# YOUR CODE HERE - 1
# Download the dataset
dataset = load_dataset("poem_sentiment")

Generating train split:   0%|          | 0/892 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/105 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/104 [00:00<?, ? examples/s]

Dataset poem_sentiment downloaded and prepared to /root/.cache/huggingface/datasets/poem_sentiment/default/1.0.0/4e44428256d42cdde0be6b3db1baa587195e91847adabf976e4f9454f6a82099. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [23]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 892
    })
    validation: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 105
    })
    test: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 104
    })
})

In [28]:
def truncate(example):
    return {
        'text': " ".join(example['verse_text'].split()[:50]),
        'label': example['label']
    }

# Take 128 random examples for train and 32 validation
small_dataset = DatasetDict(
    train=dataset['train'].shuffle(seed=1111).select(range(128)).map(truncate),
    val=dataset['train'].shuffle(seed=1111).select(range(128, 160)).map(truncate),
    test=dataset['train'].shuffle(seed=1111).select(range(160, 192)).map(truncate),
)

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [29]:
small_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'verse_text', 'label', 'text'],
        num_rows: 128
    })
    val: Dataset({
        features: ['id', 'verse_text', 'label', 'text'],
        num_rows: 32
    })
    test: Dataset({
        features: ['id', 'verse_text', 'label', 'text'],
        num_rows: 32
    })
})

In [27]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# YOUR CODE HERE - 2
# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [30]:
#['verse_text']
# YOUR CODE HERE - 3
# Set train/test/validation

small_tokenized_dataset = small_dataset.map(
    lambda example: tokenizer(example['verse_text'], truncation=True),
    batched=True,
    batch_size=16
)
train_data = None
val_data = None
test_data = None

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [31]:
from transformers import TrainingArguments, Trainer
import numpy as np

# You can optionally use this function with the right parameter to compute results.
# You don't have to use this though. You can do the caculation anyway you like.
def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return {"accuracy": np.mean(predictions == labels)}

# YOUR CODE HERE - 4
# create trainer

arguments = TrainingArguments(
    output_dir="sample_hf_trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    load_best_model_at_end=True,
    seed=224
)

trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [32]:
!mkdir sample_hf_trainer

mkdir: cannot create directory ‘sample_hf_trainer’: File exists


In [33]:
from transformers import TrainerCallback, EarlyStoppingCallback

class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
        
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")


trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=1, early_stopping_threshold=0.0))
trainer.add_callback(LoggingCallback("sample_hf_trainer/log.jsonl"))

In [35]:
device = "cuda:0"

In [38]:
# YOUR CODE HERE - 5
# Train
trainer.train()

RuntimeError: ignored

In [ ]:
# YOUR CODE HERE - 6
# Evalute Test data and report results
results = trainer.predict(small_tokenized_dataset['test'])